# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [7]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木头栈道穿过郁郁葱葱的草地，通往远方的景色。画面中的栈道非常直，似乎引领观者进入这片自然风光之中。四周的草地茂密，绿色植物生机勃勃，配合着蓝天和散布的白云，展现出一种平和而宁静的氛围。这样的场景可能位于一个公园或自然保护区，是很受欢迎的散步和观赏自然的地点。\n\n整体上，这幅图充满了夏日的氛围，色彩鲜明且分布均衡，非常适合作为自然和户外活动类的插图或背景。此外，这样的环境也可能吸引各种野生动物和鸟类，成为生态观察的好地方。', role='assistant', function_call=None, tool_calls=None))


In [8]:
response.choices[0].message.content

'这幅图展示了一条木头栈道穿过郁郁葱葱的草地，通往远方的景色。画面中的栈道非常直，似乎引领观者进入这片自然风光之中。四周的草地茂密，绿色植物生机勃勃，配合着蓝天和散布的白云，展现出一种平和而宁静的氛围。这样的场景可能位于一个公园或自然保护区，是很受欢迎的散步和观赏自然的地点。\n\n整体上，这幅图充满了夏日的氛围，色彩鲜明且分布均衡，非常适合作为自然和户外活动类的插图或背景。此外，这样的环境也可能吸引各种野生动物和鸟类，成为生态观察的好地方。'

### 封装成一个函数 query_image_description

In [9]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [13]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一张带有幽默感的比较图，展现了两种不同风格的狗狗。左边的狗狗被戏称为“16岁的我”，其形象被处理成一只头部是狗的肌肉发达的人形身体，看起来非常强壮和有力。右边的狗狗被标记为“工作后的我”，其形象是一只表情慵懒、体型稍显臃肿的普通狗狗。

图中还包括了一些幽默的文字描述：
- 左边狗狗的描述包括“我就是一个光明，每体校园，外表帅气，人见人羡的大帅比”。
- 右边狗狗的描述则是“好像有点烦，好想赖着床，蜷成一团不想动，锻炼都是骗人，负担越来越重”。

这幅图


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [21]:
from openai import OpenAI
import base64
import requests
import json
import os

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    base_url = os.environ.get("OPENAI_BASE_URL")
    response = requests.post(base_url + "/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [22]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图展示的是1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）对比。图中的纵轴代表GDP的数量（以万亿美元计），横轴则表示年份。

- 蓝线代表美国的GDP，可以看到持续上升，显示出美国经济的稳健增长。
- 红线代表中国的GDP，从图中可以清楚地看到从1990年代中期开始，中国的GDP增长速度非常快，尤其是进入21世纪后，增长更是显著，逐渐接近美国。
- 紫线显示的是日本的GDP，在1990年左右达到顶峰后有所波动，但整体趋势相对平稳。
- 绿线代表德国的GDP，增长相对平缓，但也显示出一定的增长趋势。

总的来说，这幅图很好地描绘了这四个国家在过去四十年的经济发展轨迹。可以看出中国的快速崛起，以及美国的持续领先地位。同时也反映了日本经济的长期停滞和德国相对稳定的经济表现。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [24]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片展示的是一些笔记，内容涉及到机器学习中的prompt tuning和LoRA技术。

1. **Prompt Tuning**（提示调整）: 这是一种调整小型模型（例如小型Transformer模型）的方法，通过前置的提示（Prompt）来引导模型的预测。这里列出了如何通过X（输入的嵌入向量）和权重矩阵W来计算输出Y。

2. **Prefix Tuning**（前缀调整）: 提到使用特定的前缀向量Wp和W来结合处理X，进而生成Y，适用于Transformer类的模型。

3. **LoRA**（Low-rank Adaptation，低秩适应）: 这是另一种模型调整技术，通过修改权重矩阵W的具体形式（W+ΔW），其中ΔW由低秩矩阵A和B的乘积定义，来影响模型的行为。

4. 还提到了**Q LoRA**和**LAMA**，这表明笔记的作者在探索各种高级调整和优化策略，以提高模型性能并减少模型占用的存储空间。例如，"LAMA-65GB，QLoRA-48GB" 可能指的是通过这些技术优化后模型的大小。

这些笔记反映了一位研究者或学生正在学习并应用最新的机器学习技术来改进神经网络模型。这类知识通常用于提升大型语言模型（如GPT、BERT等）的效率和性能。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [26]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

此图显示的是一个笔记本上的笔记内容，主要涉及到自然语言处理（NLP）和机器学习特别是关于变换模型（Transformers）的一些高级技术和方法。笔记的内容包括：

1. 笔记的左侧部分：
    - 提及了“变换模型”、“性能评估”和“基准测试（Benchmark）”。
    - 提到了不同的Prompt Tuning方法，如：Adapter (2019, Google), Prefix (2021, Stanford), Prompt (2021, Google)等。
    - 还有提到了一些特定的技术和模型，如L2T（Learning to Teach）、Claude、ALPHA、数据集如GPT、XLHF、ChatGPT、MOSS等。

2. 笔记的右侧部分：
    - 讨论了多模态指令型finetuning (Finetune LLMs)以及LLaMA (3B)模型。
    - 引入了新的技术和改进，如LoRA、QLoRA、AdapLoRA。
    - 讨论了“Prefix-tuning & Adapters”的不同实现，包括投影下降、非线性投影上升和加入类型。
    - 描述了适配器的高级结构，如MAM Adapters和它们如何与不同功能如FFN（Feedforward Network）集成。

总的来说，这些笔记覆盖了一系列复杂的机器学习技术，特别是在机器学习模型个性化和微调方面的最新进展。这些笔记看起来是为了学习或研究目的而记录的，其中包含了许多专业术语和概念，适用于对AI技术有深入了解的读者。

![](./images/handwriting_1.jpg)

In [27]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片显示的是一份关于自然语言处理和机器学习技术的笔记。笔记中列出了多种技术和概念，重点是Transformers模型的不同适配和调整方法。

左边部分的笔记中提及了：
1. Transformers和状态（State-of-the-art, SOTA）技术的比较及基准。
2. PEFT (可能指某种特定的Fine-tuning技术) 和PBFT Methods。
3. Prompt Tuning, 包括不同年份和来源的方法，如Adaptor (2019, Google), Prefix (2021, Stanford), 和Prompt (2021, Google)。
4. 提到了一些具体的模型，如Llama（2021）、ChatGPT、Bloom、和Alpaca。

右侧部分的笔记提到：
1. 多模态指令微调技术（multi-modality instruction fine-tuning)，例如LLaMA (3B)。
2. 提到了新技术，如LoRA和PETC（2022），再拓展至具体的技术改变，如GLoRA和Adalora。
3. 对于Prefix-tuning和Adaptors的技术探讨较多，提到了它们在LLMs中的应用以及操作原理，如project down、non-learn、project up、insertion form等。

此图片提供了关于先进机器学习模型调整技术的快照，适合深入研究自然语言处理或机器学习的技术人员参考。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

![](./images/handwriting_lam.jpg)

In [28]:
content = query_base64_image_description("./images/handwriting_lam.jpg", prompt="帮我解析内容是什么并解释一下内容")
print(content)

这张图片包含了中英文混合的手写文字，首先是中文手写部分，看起来像是关于某个日常记录或思考，但因为部分笔迹不清楚，造成一些文字难以完全确认。此外，图片下方写有英文：“This is for Lambert Lin testing content in ChatGPT-HD, content contains English and Chinese.”这句话意味这张纸是为了测试Lambert Lin在ChatGPT-HD中的内容，且内容包含中文和英文。

如果需要更准确地翻译或理解中文部分的具体内容，可能需要更清晰的图片或其他帮助。但整体来看，图片似乎用于某种测试目的，涉及到多语言处理。


## Homework: 
### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。